In [1]:
import torch
import os
import numpy as np
from Hang.utils_u_groupnorm_pytorchLightning import *
from utils import *
import time as time
import nibabel as nib
import random
from tqdm import tqdm

np.random.seed(0)
random.seed(5)
torch.manual_seed(0)

/data/Jeremy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/data/Jeremy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/data/Jeremy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/data/Jeremy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
numbers = ["0001","0017","0018","0038","0040","0042","0046","0087","0090","0108","0116","0131","0178","0190",
           "0227","0248","0267","0282","0285","0398","0448","0466","0504","0514","0535","0564","0598","0606",
           "0607","0618","0620","0623","0642","0646","0655","0668","0675","0681","0719","0761","0762","0783",
           "0786","0868","0877","0887","0895","0902","0931","0979","1007","1013","1029","1033","1068","1142",
           "1143","1163","1190","1260","1275","1347","1383","1389","1416","1435","1441","1447","1451","1514",
           "1520","1602","1611","1621","1680","1684","1686","1710","1720","1739","1743","1749","1753","1760",
           "1795","1805","1845","1858","1876","1889","1892","1898","1899","1918","1924","1932","1952","1961",
           "1972","1987","2003","2007","2016","2020","2022","2030","2045","2047","2049","2053","2055","2074",
           "2077","2080","2091","2094","2103","2115","2128","2142","2144","2146","2152","2156","2158","2160",
           "2161","2179","2180","2181","2183","2186","2188","2212","2221","2231","2234","2245"]

In [3]:
numbers = [numbers[-1]]

### Generate noise

In [5]:
SNR = 200

In [6]:
roi_subjects = ["0017", "0018", "0038", "0040", "0042", "0046", "0087"]
roi_files = []
roi_fastt2_files = []
for subject in roi_subjects:
    roi_files.append(f"../{subject}/roi.nii.gz")
    roi_fastt2_files.append(f"../{subject}/FASTT2_FULL.nii.gz")
rois = []
fastt2 = [[] for i in range(6)]
for file in roi_files:
    rois.append(nib.load(file).get_fdata())
for idx in range(len(roi_fastt2_files)):
    brain = nib.load(roi_fastt2_files[idx]).get_fdata()
    for i in range(6):
        te = brain[:,:,:,i]
        fastt2[i] += te[rois[idx] == 1].tolist()

In [7]:
rois = np.array(rois)
fastt2 = np.array(fastt2)

In [8]:
NOISE_SCALE = fastt2[0].mean()/SNR
print(NOISE_SCALE)

4.704228281640453


### Grab 256x256 brains

In [12]:
def generateBrains(brainRef, labels, masks, noise):
    label = labels[0].copy()
    brains = np.zeros((NUM_BRAINS,6)+label[0].shape)
#     echoTimes = [10 * i for i in range(33)]
#     echoTimes = [0, 7.5, 17.5, 67.5, 147.5, 307.5]
    echoTimes = [10, 20, 40, 80, 160, 320]
    for labelIdx in range(len(labels)):
        label = labels[labelIdx].copy()
        for i in range(len(echoTimes)):
            brains[labelIdx,i] = label[0] * np.exp(-echoTimes[i] / (label[3] + 1e-16)) + \
                                 label[1] * np.exp(-echoTimes[i] / (label[4] + 1e-16)) + \
                                 label[2] * np.exp(-echoTimes[i] / (label[5] + 1e-16))
            np.nan_to_num(brains[labelIdx,i], copy=False)
        scale = brainRef[labelIdx,0] / (brains[labelIdx,0] + 1e-16)
        for i in range(len(echoTimes)):
            brains[labelIdx,i] *= scale
    brains += noise
    for i in range(len(echoTimes)):
        brains[:,i] *= masks
#     brains = brains[:,1:]
    print(brains.shape)
    return brains

In [11]:
batch = 1
for q in range(0, len(numbers), batch):
    fastt2_files, mask_files = [], []
    w1_files, w2_files, w3_files = [], [], []
    t1_files, t2_files, t3_files = [], [], []

    for number in numbers[q:q+batch]:
        file_root = "../" + number + "/"
        mask_files.append(file_root + "tightmask.nii.gz")
        fastt2_files.append(file_root + "FASTT2_FULL.nii.gz")
        w1_files.append(file_root + "w1.nii.gz")
        w2_files.append(file_root + "w2.nii.gz")
        w3_files.append(file_root + "w3.nii.gz")
        t1_files.append(file_root + "t1.nii.gz")
        t2_files.append(file_root + "t2.nii.gz")
        t3_files.append(file_root + "t3.nii.gz")

    mask_array = []
    brains = []
    labels = []

    for i in range(len(mask_files)):
        mask_array.append(nib.load(mask_files[i]).get_fdata())

    for i in range(len(mask_files)):
        brain = nib.load(fastt2_files[i]).get_fdata().transpose((3,0,1,2)) * mask_array[i]
        brains.append(brain)

    for i in range(len(mask_files)):
        label = []
        label.append(nib.load(w1_files[i]).get_fdata() * mask_array[i])
        label.append(nib.load(w2_files[i]).get_fdata() * mask_array[i])
        label.append(nib.load(w3_files[i]).get_fdata() * mask_array[i])
        label.append(nib.load(t1_files[i]).get_fdata() * mask_array[i])
        label.append(nib.load(t2_files[i]).get_fdata() * mask_array[i])
        label.append(nib.load(t3_files[i]).get_fdata() * mask_array[i])
        labels.append(label)

    # BIG NUMBER: n, 6, 256, 256, 32
    brains = np.array(brains)
    labels = np.array(labels)
    mask_array = np.array(mask_array)

    NUM_BRAINS = len(numbers[q:q+batch])
    noise = np.random.normal(size=(NUM_BRAINS,6) + labels[0,0].shape, scale=NOISE_SCALE)
    simbrains_200 = generateBrains(brains, labels, mask_array, noise)
    for i in range(len(simbrains_200)):
        newbrain = simbrains_200[i]
        number = numbers[q:q+batch][i]
        save_nii(newbrain.transpose((1,2,3,0)), f"../{number}/FASTT2_SNR_200_SIM_NEW_TE.nii.gz", f"../{number}/w1.nii.gz")

(1, 6, 256, 256, 32)
